# Import Required Libraries

In [73]:
import keras
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
from keras.datasets import mnist
from pandas import DataFrame, read_csv
import random
import os
import math
from keras.callbacks import Callback

USE_GPU = True
if USE_GPU:
    print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [81]:
data_df = read_csv('final/modified-data/bank_categorical_translate.csv')
training_data_df = data_df.sample(math.ceil(len(data_df.index)*0.8))
df_all = data_df.merge(training_data_df.drop_duplicates(),
                    on=list(data_df.columns), 
                   how='left',
                   indicator=True
                    )

df_all = df_all.loc[df_all['_merge'] == 'left_only']
validation_data_df = df_all.drop('_merge', axis=1)
training_target_df = training_data_df.pop('target')
validation_target_df = pd.DataFrame(validation_data_df.pop('target')) # drop this column, but also return a dataframe

# Construct the Network

In [76]:
# assemble the fully connected network
number_of_columns = training_data_df.shape[1]
model = Sequential()

model.add(keras.layers.Dense(64, activation="LeakyReLU", input_shape=(number_of_columns,)))

# add the fully connected layers in a funnel shape. Try it out!
for layer_size in [64, 32, 16, 8, 4, 2]:
    model.add(keras.layers.Dense(layer_size, activation='LeakyReLU'))
    # model.add(keras.layers.Dropout(.05, noise_shape=None, seed=None))
    # model.add(keras.layers.BatchNormalization())

model.add(Flatten())
model.add(keras.layers.Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 64)                3264      
                                                                 
 dense_25 (Dense)            (None, 64)                4160      
                                                                 
 dense_26 (Dense)            (None, 32)                2080      
                                                                 
 dense_27 (Dense)            (None, 16)                528       
                                                                 
 dense_28 (Dense)            (None, 8)                 136       
                                                                 
 dense_29 (Dense)            (None, 4)                 36        
                                                                 
 dense_30 (Dense)            (None, 2)                

# Train the Neural Network

In [77]:
BATCH_SIZE = 2

model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

print("Starting Training.")

history = model.fit(
    training_data_df,
    training_target_df,
    validation_data=(validation_data_df, validation_target_df),
    batch_size=BATCH_SIZE,
    epochs=1
)

print("Training is complete.")

Starting Training.
16476/16476 [==============================] - 273s 16ms/step - loss: 0.2906 - accuracy: 0.8953 - val_loss: 0.2933 - val_accuracy: 0.8906
Training is complete.


In [78]:
print("Starting Testing.")
scores = model.evaluate(validation_data_df, validation_target_df)
print("Testing is Complete.")
print("test loss, test accuracy:", scores)

# configure for validation section
result = list(model.predict(validation_data_df))
validation_target_df['result'] = result

Starting Testing.
242/242 [==============================] - 2s 9ms/step - loss: 0.2933 - accuracy: 0.8906
Testing is Complete.
test loss, test accuracy: [0.2933323383331299, 0.890602707862854]


In [93]:
TP: int = 0
TN: int = 0
FP: int = 0
FN: int = 0

for idx, row in validation_target_df.iterrows():

    result = float(row['result'])
    target = row['target']

    # predicted: positive
    if result > 0.5:
        
        # true positive 
        if target == 1: TP += 1
        
        # false positive
        else: FP += 1
    
    # predicted: negative
    else:

        # true negative
        if target == 0: TN += 1

        # false negative
        else: FN += 1

print(
f'''true positives: {TP}
true negatives: {TN}
false positives: {FP}
false negatives: {FN}
''')

# determine the accuracy, precision, and recall
accuracy = (TP + TN) / (TN + FN + TP + FP)
precision = TP / (TP + FP)
recall = TP / (TP +FN)

print(f'''
accuracy: {accuracy * 100}
precision: {precision * 100}
recall: {recall * 100}
''')



true positives: 110
true negatives: 6761
false positives: 24
false negatives: 809


accuracy: 89.18743509865004
precision: 82.08955223880598
recall: 11.969532100108813

